In [83]:
import pandas as pd
import numpy as np
import json
import pickle
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kratikakothari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kratikakothari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
# Preprocessing the dataset
# The dataset is kaggle and nature dataset
# It contains set of ingredients and the associated cuisine for each recipe.
f_kaggleNature = open('kaggle_and_nature.csv', newline = '')    
csv_reader = csv.reader(f_kaggleNature, delimiter='\t')

id_ingredients_cuisine = []
cuisines = []
i = 0
for row in csv_reader:
    temp = dict()
    temp['id'] = i
    ingredients = []
    for ingredient in row[0].split(",")[1:]:
        ingredients.append(ingredient.replace("_"," "))
    temp['ingredients'] = ingredients
    temp['cuisine'] = row[0].split(",")[0]
    id_ingredients_cuisine.append(temp)
    cuisines.append(game[0].split(",")[0])
    i = i + 1
id_ingredients_cuisine

[{'id': 0, 'ingredients': [], 'cuisine': '#'},
 {'id': 1,
  'ingredients': [],
  'cuisine': '# Flavor network and the principles of food pairing'},
 {'id': 2,
  'ingredients': [' Sebastian E. Ahnert',
   ' James P. Bagrow',
   ' and Albert-Laszlo Barabasi'],
  'cuisine': '# Yong-Yeol Ahn'},
 {'id': 3, 'ingredients': [], 'cuisine': '# '},
 {'id': 4,
  'ingredients': ['chicken', 'cinnamon', 'soy sauce', 'onion', 'ginger'],
  'cuisine': 'African'},
 {'id': 5,
  'ingredients': ['cane molasses',
   'ginger',
   'cumin',
   'garlic',
   'tamarind',
   'bread',
   'coriander',
   'vinegar',
   'onion',
   'beef',
   'cayenne',
   'parsley',
   'wheat bread',
   'yogurt',
   'vegetable oil',
   'egg'],
  'cuisine': 'African'},
 {'id': 6,
  'ingredients': ['butter',
   'pepper',
   'onion',
   'cardamom',
   'cayenne',
   'ginger',
   'cottage cheese',
   'garlic',
   'brassica'],
  'cuisine': 'African'},
 {'id': 7,
  'ingredients': ['olive oil',
   'pepper',
   'wheat',
   'beef',
   'onion',


In [85]:
# Removing Punctuation, Stopwords
# Lemmatization

stop_words = set(stopwords.words('english'))
intab = '''!()-[]{};:'"\,<>?@#$%^&*_~'''
outtab = "_" * len(intab)
trantab = str.maketrans(intab, outtab)

lemmatizer = WordNetLemmatizer()

new_id_ingredients_cuisine = []
max_ingredients = 0

for recipe_id,recipe in enumerate(id_ingredients_cuisine):
    temp_ingredients = []
    for ingredient in recipe["ingredients"]:
        word_tokens = word_tokenize(ingredient)
        l = []
        for word in word_tokens:
            if word not in stop_words:
                word = word.translate(trantab).replace("_", "").lower()
                l.append(lemmatizer.lemmatize(word))
        ingredient_modified = " ".join(l)
        temp_ingredients.append(ingredient_modified)
    temp = dict()
    temp['id'] = recipe_id
    temp['ingredients'] = sorted(temp_ingredients)
    temp['cuisine'] = recipe["cuisine"]
    new_id_ingredients_cuisine.append(temp)
    if(len(temp_ingredients) > max_ingredients):
        max_ingredients  = len(temp_ingredients)
new_id_ingredients_cuisine

[{'id': 0, 'ingredients': [], 'cuisine': '#'},
 {'id': 1,
  'ingredients': [],
  'cuisine': '# Flavor network and the principles of food pairing'},
 {'id': 2,
  'ingredients': ['albertlaszlo barabasi',
   'james p. bagrow',
   'sebastian e. ahnert'],
  'cuisine': '# Yong-Yeol Ahn'},
 {'id': 3, 'ingredients': [], 'cuisine': '# '},
 {'id': 4,
  'ingredients': ['chicken', 'cinnamon', 'ginger', 'onion', 'soy sauce'],
  'cuisine': 'African'},
 {'id': 5,
  'ingredients': ['beef',
   'bread',
   'cane molasses',
   'cayenne',
   'coriander',
   'cumin',
   'egg',
   'garlic',
   'ginger',
   'onion',
   'parsley',
   'tamarind',
   'vegetable oil',
   'vinegar',
   'wheat bread',
   'yogurt'],
  'cuisine': 'African'},
 {'id': 6,
  'ingredients': ['brassica',
   'butter',
   'cardamom',
   'cayenne',
   'cottage cheese',
   'garlic',
   'ginger',
   'onion',
   'pepper'],
  'cuisine': 'African'},
 {'id': 7,
  'ingredients': ['beef',
   'cardamom',
   'cumin',
   'garlic',
   'leek',
   'olive 

In [86]:
corpus = []
for recipe_id,recipe in enumerate(new_id_ingredients_cuisine):
    corpus.append(recipe["ingredients"])
        
# Training word2vec model
        
model = Word2Vec(corpus, min_count = 1, size = 100, workers = 2, window = max_ingredients, sg = 0)

model.save("word2vec.model")
model.wv.save_word2vec_format('word2vec.txt', binary=False)




/usr/local/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [87]:
import matplotlib.pyplot as plt 
from sklearn.manifold import TSNE

In [89]:
set_ingredients = set()
for recipe_id,recipe in enumerate(new_id_ingredients_cuisine):
    set_ingredients.update(recipe['ingredients'])


In [ ]:
XY = TSNE.fit_transform(model) 
# textscatter(XY,list(set_ingredients))
X = XY[:, 0]
Y = XY[:, 1]
fig=plt.figure()
plt.scatter(X,Y)